# `torch.autograd` 쉽게 배우기

`torch.autograd`는 신경망을 학습할 때 자동으로 미분(gradient)을 계산해주는 PyTorch의 기능입니다. 이 덕분에 복잡한 수식을 직접 계산하지 않아도 됩니다.

**이 튜토리얼에서 배우는 것**

- 순전파/역전파가 무엇인지 한 문장으로 이해하기
- `.backward()`가 무엇을 하는지, 언제 `gradient` 인자가 필요한지 알기
- `.grad`에 저장된 기울기를 사용해 파라미터를 업데이트하는 흐름 익히기

**배경 설명**

신경망(Neural Network, NN)은 여러 함수(레이어)로 구성되어 있고, 각 함수는 가중치와 편향 같은 *파라미터*를 가지고 있습니다. 이 값들은 PyTorch에서 텐서(tensor)로 저장됩니다.

신경망 학습은 두 단계로 이루어집니다:

- **순전파(Forward Propagation)**: 입력 데이터를 신경망에 넣어 예측값을 만듭니다.
- **역전파(Backward Propagation)**: 예측값과 실제값의 차이(오차)를 계산하고, 오차가 각 파라미터에 얼마나 영향을 주는지(기울기, gradient)를 자동으로 계산합니다. 이 기울기를 이용해 파라미터를 조금씩 수정합니다.

자세한 역전파 설명은 [3Blue1Brown의 영상](https://www.youtube.com/watch?v=tIeHLnjs5U8)에서 확인할 수 있습니다.

## PyTorch에서의 사용 예시

아래 예시는 미리 학습된 ResNet18 모델을 불러와 임의의 이미지를 입력하고, 예측값과 실제값의 차이를 계산한 뒤, 역전파로 파라미터의 기울기를 구하는 과정입니다.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>참고:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>이 튜토리얼은 CPU에서만 동작하며, GPU에서는 실행되지 않습니다.</p>

</div>


In [32]:
import torch  # PyTorch 불러오기
from torchvision.models import resnet18, ResNet18_Weights  # torchvision에서 ResNet18과 가중치 불러오기
model = resnet18(weights=ResNet18_Weights.DEFAULT)  # 사전학습된(미리 학습된) ResNet18 모델 생성

data = torch.rand(1, 3, 64, 64)  # 배치 크기=1, 채널=3(RGB), 이미지 크기=64x64인 임의 입력
labels = torch.rand(1, 1000)      # 임의의 타깃(분류 1000차원). 실제 학습에선 정답 레이블을 사용합니다

이제 입력 데이터를 모델에 넣어 각 레이어를 통과시키며 예측값을 만듭니다. 이것이 바로 **순전파**입니다.


In [33]:
# 순전파: 입력을 모델에 넣어 예측값을 계산합니다.
prediction = model(data)
# prediction의 형태는 (배치크기, 클래스 수)입니다. 여기서는 (1, 1000).

모델의 예측값과 정답(label)을 이용해 오차(`loss`)를 계산합니다. 다음 단계는 이 오차를 신경망 전체에 역전파하는 것입니다. 오차 텐서에서 `.backward()`를 호출하면 autograd가 각 파라미터의 기울기를 자동으로 계산해 `.grad`에 저장합니다.


In [34]:
# 간단한 손실(loss) 정의: 예측과 정답의 차이를 모두 더합니다.
# 실제로는 CrossEntropyLoss 같은 적절한 손실함수를 사용합니다.
loss = (prediction - labels).sum()
# 역전파: 오차를 이용해 파라미터의 기울기를 계산합니다.
loss.backward()

다음으로 옵티마이저(optimizer)를 불러옵니다. 여기서는 SGD(확률적 경사하강법)를 사용하며, 학습률은 0.01, 모멘텀(momentum)은 0.9로 설정합니다. 모델의 모든 파라미터를 옵티마이저에 등록합니다.


In [35]:
# 옵티마이저 생성: 모델의 모든 파라미터를 등록
# lr=1e-2는 학습률, momentum=0.9는 최근 기울기를 참고해 더 안정적으로 이동하게 해줍니다.
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

마지막으로 `.step()`을 호출하면 경사하강법이 시작됩니다. 옵티마이저는 각 파라미터를 `.grad`에 저장된 기울기만큼 조정합니다.


In [36]:
# 경사하강법: 파라미터를 업데이트합니다.
# 일반적인 학습 루프에서는 먼저 optim.zero_grad()로 이전 step의 기울기를 초기화한 뒤,
# loss.backward()로 현재 배치의 기울기를 계산하고, optim.step()으로 업데이트합니다.
optim.step()

## Autograd에서의 미분

autograd가 어떻게 기울기를 계산하는지 살펴봅시다. `requires_grad=True`로 생성한 두 텐서 `a`와 `b`를 만듭니다. 이렇게 하면 autograd가 모든 연산을 추적합니다.

In [37]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True) # 두 텐서 모두 autograd가 추적합니다.

이제 `a`와 `b`를 이용해 새로운 텐서 `Q`를 만듭니다.

$$Q = 3a^3 - b^2$$


In [38]:
Q = 3*a**3 - b**2 # Q는 a, b로부터 만들어진 텐서입니다.

`a`와 `b`가 신경망의 파라미터이고, `Q`가 오차라고 생각해봅시다. 신경망 학습에서는 오차에 대한 파라미터의 기울기를 구합니다.

$$\frac{\partial Q}{\partial a} = 9a^2$$
$$\frac{\partial Q}{\partial b} = -2b$$

`Q.backward()`를 호출하면 autograd가 이 기울기를 계산해 각각의 `.grad`에 저장합니다.

여기서 중요한 점: `Q`가 벡터이므로 그대로는 역전파를 시작할 수 없습니다. 체인룰의 시작점이 되는 "스칼라"가 없기 때문이죠. 그래서 `Q.backward(gradient=...)`에서 `gradient`를 넘겨 줍니다.

- `gradient`의 의미: `d(외부스칼라)/dQ`이며, 보통 `torch.ones_like(Q)`로 두면 각 요소를 동일 가중치로 합한 스칼라를 가정하는 것과 같습니다.
- 다른 비율의 가중치가 필요하면, 예: `torch.tensor([0.7, 0.3])`처럼 요소별 중요도를 반영할 수 있습니다.

In [39]:
# Q는 길이 2의 벡터이므로 backward에 gradient 인자가 필요합니다.
# 가장 흔한 선택은 각 요소를 동일 비율로 합친다고 가정하는 ones 텐서입니다.
external_grad = torch.ones_like(Q)
Q.backward(gradient=external_grad)  # 기울기 계산

또는 Q를 합쳐서 스칼라로 만든 뒤 `Q.sum().backward()`를 호출해도 됩니다. 이 경우에는 `gradient` 인자가 필요 없습니다.

주의: 같은 그래프에서 `backward()`를 여러 번 호출하면 기울기가 누적(accumulate)됩니다. 매 미니배치마다 학습한다면, 일반적으로 `optim.zero_grad()`를 먼저 호출해 이전 기울기를 초기화한 뒤 `loss.backward()`를 호출하세요.

In [42]:
# (대안) 스칼라로 합쳐서 backward 하는 방법
# 두 번째 backward를 보여주기 전에, 이전에 쌓인 기울기를 초기화합니다.
a.grad = None
b.grad = None
# 이전 backward로 그래프가 해제되었으므로, Q를 다시 계산해 새 그래프를 만듭니다.
Q = 3*a**3 - b**2
Q.sum().backward()  # 기울기 계산
# 현재 a=[2,3], b=[6,4] 이므로 이론값은:
# 9*a**2 = tensor([36., 81.])
# -2*b    = tensor([-12.,  -8.])
print(9*a**2 == a.grad)
print(-2*b == b.grad)
print("a.grad:", a.grad)
print("b.grad:", b.grad)

tensor([True, True])
tensor([True, True])
a.grad: tensor([36., 81.])
b.grad: tensor([-12.,  -8.])
